<a href="https://colab.research.google.com/github/sabvz-run/solarium_siep/blob/main/notebooks/2_solarium_siep_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#═══════════════════════════════════════════════════════════════
# ETAPA 1: INSTALAÇÃO DE BIBLIOTECAS NECESSÁRIAS
#═══════════════════════════════════════════════════════════════
# Instalar bibliotecas que não vêm por padrão no Colab

!pip install scikit-fuzzy xgboost lightgbm -q

print("Bibliotecas instaladas com sucesso!")

Bibliotecas instaladas com sucesso!


In [3]:
#═══════════════════════════════════════════════════════════════
# ETAPA 2: IMPORTAÇÃO DE BIBLIOTECAS
#═══════════════════════════════════════════════════════════════

# Manipulação de dados
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Pré-processamento
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Modelos de Machine Learning
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb

# Lógica Fuzzy
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Suprimir warnings desnecessários
import warnings
warnings.filterwarnings('ignore')

print("Todas as bibliotecas importadas com sucesso!")

Todas as bibliotecas importadas com sucesso!


In [4]:
#═══════════════════════════════════════════════════════════════
# ETAPA 3: CARREGAMENTO DO DATASET SUPREMO
#═══════════════════════════════════════════════════════════════

# URL do dataset supremo no GitHub
url_dataset = 'https://raw.githubusercontent.com/sabvz-run/solarium_siep/main/datasets/dataset_supremo.csv'

print("Carregando dataset supremo do GitHub...")

# Carregar dataset
df = pd.read_csv(url_dataset)

# Converter coluna de data para datetime
df['data_hora'] = pd.to_datetime(df['data_hora'])

# Informações básicas
print(f"\nDataset carregado com sucesso!")
print(f"Total de registros: {len(df):,}")
print(f"Total de colunas: {len(df.columns)}")
print(f"Período: {df['data_hora'].min()} até {df['data_hora'].max()}")
print(f"\nColunas disponíveis:\n{df.columns.tolist()}")

# Mostrar primeiras linhas
print("\nPrimeiras linhas do dataset:")
df.head()

Carregando dataset supremo do GitHub...


HTTPError: HTTP Error 404: Not Found

In [ ]:
#═══════════════════════════════════════════════════════════════
# ETAPA 4: ANÁLISE E TRATAMENTO DE VALORES AUSENTES
#═══════════════════════════════════════════════════════════════

print("Analisando valores ausentes...")

# Contar NaN por coluna
valores_ausentes = df.isnull().sum()
percentual_ausentes = (valores_ausentes / len(df)) * 100

# Criar DataFrame com informações
info_ausentes = pd.DataFrame({
    'Coluna': valores_ausentes.index,
    'Valores_Ausentes': valores_ausentes.values,
    'Percentual': percentual_ausentes.values
})

# Filtrar apenas colunas com valores ausentes
info_ausentes = info_ausentes[info_ausentes['Valores_Ausentes'] > 0].sort_values('Valores_Ausentes', ascending=False)

print("\nColunas com valores ausentes:")
print(info_ausentes.to_string(index=False))

# Identificar colunas críticas para o modelo
colunas_criticas = ['geracao_kw', 'temperatura_ambiente', 'irradiacao']

print(f"\nColunas críticas para o modelo: {colunas_criticas}")

In [ ]:
#═══════════════════════════════════════════════════════════════
# ETAPA 5: LIMPEZA E PREPARAÇÃO DOS DADOS
#═══════════════════════════════════════════════════════════════

print("Iniciando processo de limpeza...")

# Estratégia 1: Remover registros sem geração (target)
df_limpo = df.dropna(subset=['geracao_kw']).copy()
print(f"Após remover registros sem geração: {len(df_limpo):,} registros")

# Estratégia 2: Para colunas climáticas, usar interpolação temporal
# Isso preenche gaps usando valores vizinhos (melhor que deletar)
colunas_interpolar = ['temperatura_ambiente', 'irradiacao', 'temperatura_painel']

for coluna in colunas_interpolar:
    if coluna in df_limpo.columns:
        # Interpolação temporal (considera a ordem cronológica)
        df_limpo[coluna] = df_limpo[coluna].interpolate(method='time', limit=3)
        print(f"Interpolação aplicada em: {coluna}")

# Estratégia 3: Remover registros que ainda têm NaN nas colunas críticas
df_limpo = df_limpo.dropna(subset=colunas_criticas)
print(f"\nDataset final limpo: {len(df_limpo):,} registros")
print(f"Registros removidos: {len(df) - len(df_limpo):,} ({((len(df) - len(df_limpo))/len(df)*100):.2f}%)")

# Verificar se ainda há NaN
nan_restantes = df_limpo.isnull().sum().sum()
print(f"Valores NaN restantes: {nan_restantes}")

# Resetar índice
df_limpo.reset_index(drop=True, inplace=True)

print("\nLimpeza concluída!")

In [ ]:
#═══════════════════════════════════════════════════════════════
# ETAPA 6: ANÁLISE EXPLORATÓRIA DE DADOS (EDA)
#═══════════════════════════════════════════════════════════════

print("Gerando estatísticas descritivas...")

# Estatísticas das principais variáveis
colunas_analisar = ['geracao_kw', 'irradiacao', 'temperatura_ambiente']
estatisticas = df_limpo[colunas_analisar].describe()

print("\nEstatísticas Descritivas:")
print(estatisticas)

# Visualização 1: Distribuição das variáveis principais
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(df_limpo['geracao_kw'], bins=50, color='skyblue', edgecolor='black')
axes[0].set_title('Distribuição de Geração (kW)')
axes[0].set_xlabel('Geração (kW)')
axes[0].set_ylabel('Frequência')

axes[1].hist(df_limpo['irradiacao'], bins=50, color='orange', edgecolor='black')
axes[1].set_title('Distribuição de Irradiação')
axes[1].set_xlabel('Irradiação (W/m²)')
axes[1].set_ylabel('Frequência')

axes[2].hist(df_limpo['temperatura_ambiente'], bins=50, color='salmon', edgecolor='black')
axes[2].set_title('Distribuição de Temperatura')
axes[2].set_xlabel('Temperatura (°C)')
axes[2].set_ylabel('Frequência')

plt.tight_layout()
plt.show()

print("Gráficos de distribuição gerados!")

In [ ]:
#═══════════════════════════════════════════════════════════════
# ETAPA 7: ANÁLISE DE CORRELAÇÃO ENTRE VARIÁVEIS
#═══════════════════════════════════════════════════════════════

print("Calculando correlações...")

# Selecionar apenas colunas numéricas
colunas_numericas = df_limpo.select_dtypes(include=[np.number]).columns.tolist()

# Calcular matriz de correlação
correlacao = df_limpo[colunas_numericas].corr()

# Visualizar matriz de correlação
plt.figure(figsize=(10, 8))
sns.heatmap(correlacao, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=1)
plt.title('Matriz de Correlação - Variáveis do Dataset')
plt.tight_layout()
plt.show()

# Correlação específica com a variável target (geracao_kw)
correlacao_target = correlacao['geracao_kw'].sort_values(ascending=False)
print("\nCorrelação com Geração (kW):")
print(correlacao_target)

In [ ]:
#═══════════════════════════════════════════════════════════════
# ETAPA 8: FEATURE ENGINEERING (CRIAÇÃO DE NOVAS VARIÁVEIS)
#═══════════════════════════════════════════════════════════════

print("Criando novas features...")

# Ordenar por data para criar features temporais corretamente
df_limpo = df_limpo.sort_values('data_hora').reset_index(drop=True)

# Features temporais básicas
df_limpo['hora'] = df_limpo['data_hora'].dt.hour
df_limpo['dia'] = df_limpo['data_hora'].dt.day
df_limpo['mes'] = df_limpo['data_hora'].dt.month
df_limpo['dia_semana'] = df_limpo['data_hora'].dt.dayofweek  # 0=Segunda, 6=Domingo
df_limpo['eh_fim_de_semana'] = (df_limpo['dia_semana'] >= 5).astype(int)

# Features de período do dia
df_limpo['periodo_dia'] = pd.cut(df_limpo['hora'],
                                  bins=[0, 6, 12, 18, 24],
                                  labels=['madrugada', 'manha', 'tarde', 'noite'],
                                  include_lowest=True)

# Converter período_dia para dummies (one-hot encoding)
periodo_dummies = pd.get_dummies(df_limpo['periodo_dia'], prefix='periodo')
df_limpo = pd.concat([df_limpo, periodo_dummies], axis=1)

# Lag features (valores anteriores)
# Útil para séries temporais: valor de 1h atrás pode prever o próximo
df_limpo['geracao_1h_atras'] = df_limpo['geracao_kw'].shift(1)
df_limpo['irradiacao_1h_atras'] = df_limpo['irradiacao'].shift(1)

# Rolling statistics (médias móveis)
# Captura tendências de curto prazo
df_limpo['geracao_media_6h'] = df_limpo['geracao_kw'].rolling(window=6, min_periods=1).mean()
df_limpo['irradiacao_media_3h'] = df_limpo['irradiacao'].rolling(window=3, min_periods=1).mean()

# Features de interação (multiplicação de variáveis relacionadas)
df_limpo['irrad_x_temp'] = df_limpo['irradiacao'] * df_limpo['temperatura_ambiente']

# Remover primeiras linhas com NaN criados pelos shifts
df_limpo = df_limpo.dropna().reset_index(drop=True)

print(f"\nFeatures criadas com sucesso!")
print(f"Total de colunas agora: {len(df_limpo.columns)}")
print(f"Registros finais: {len(df_limpo):,}")

In [ ]:
#═══════════════════════════════════════════════════════════════
# ETAPA 9: PREPARAÇÃO DOS DADOS PARA MODELAGEM
#═══════════════════════════════════════════════════════════════

print("Preparando dados para treinamento...")

# Definir features (X) e target (y)
# Remover colunas não utilizáveis no modelo
colunas_remover = ['data_hora', 'periodo_dia']  # data e categóricas já tratadas

# Selecionar features numéricas
X = df_limpo.drop(columns=['geracao_kw'] + colunas_remover)
y = df_limpo['geracao_kw']

print(f"\nFeatures (X): {X.shape}")
print(f"Target (y): {y.shape}")
print(f"\nFeatures utilizadas:\n{X.columns.tolist()}")

# Split treino/validação/teste (60% / 20% / 20%)
# Primeiro: separar treino+validação (80%) e teste (20%)
X_temp, X_teste, y_temp, y_teste = train_test_split(
    X, y, test_size=0.20, random_state=42, shuffle=False  # shuffle=False mantém ordem temporal
)

# Segundo: separar treino (60%) e validação (20%)
X_treino, X_val, y_treino, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, shuffle=False  # 0.25 de 80% = 20% do total
)

print(f"\nDivisão dos dados:")
print(f"Treino: {len(X_treino):,} registros ({len(X_treino)/len(X)*100:.1f}%)")
print(f"Validação: {len(X_val):,} registros ({len(X_val)/len(X)*100:.1f}%)")
print(f"Teste: {len(X_teste):,} registros ({len(X_teste)/len(X)*100:.1f}%)")

# Normalização (StandardScaler)
# Importante: fit apenas no treino, transform em todos
scaler = StandardScaler()
X_treino_scaled = scaler.fit_transform(X_treino)
X_val_scaled = scaler.transform(X_val)
X_teste_scaled = scaler.transform(X_teste)

print("\nNormalização aplicada com sucesso!")

In [ ]:
#═══════════════════════════════════════════════════════════════
# ETAPA 10: TREINAMENTO - MODELO 1: RANDOM FOREST
#═══════════════════════════════════════════════════════════════

print("Treinando Random Forest Regressor...")

# Inicializar modelo
modelo_rf = RandomForestRegressor(
    n_estimators=100,      # Número de árvores
    max_depth=15,          # Profundidade máxima
    min_samples_split=10,  # Mínimo de amostras para dividir
    random_state=42,
    n_jobs=-1              # Usar todos os núcleos do CPU
)

# Treinar
modelo_rf.fit(X_treino_scaled, y_treino)

# Predições
y_pred_rf_treino = modelo_rf.predict(X_treino_scaled)
y_pred_rf_val = modelo_rf.predict(X_val_scaled)

# Métricas no conjunto de validação
mae_rf = mean_absolute_error(y_val, y_pred_rf_val)
rmse_rf = np.sqrt(mean_squared_error(y_val, y_pred_rf_val))
r2_rf = r2_score(y_val, y_pred_rf_val)

print("\nResultados Random Forest (Validação):")
print(f"MAE (Erro Médio Absoluto): {mae_rf:.4f} kW")
print(f"RMSE (Raiz do Erro Quadrático Médio): {rmse_rf:.4f} kW")
print(f"R² (Coeficiente de Determinação): {r2_rf:.4f}")

In [ ]:
#═══════════════════════════════════════════════════════════════
# ETAPA 11: TREINAMENTO - MODELO 2: XGBOOST
#═══════════════════════════════════════════════════════════════

print("Treinando XGBoost Regressor...")

# Inicializar modelo
modelo_xgb = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1
)

# Treinar
modelo_xgb.fit(X_treino_scaled, y_treino)

# Predições
y_pred_xgb_val = modelo_xgb.predict(X_val_scaled)

# Métricas no conjunto de validação
mae_xgb = mean_absolute_error(y_val, y_pred_xgb_val)
rmse_xgb = np.sqrt(mean_squared_error(y_val, y_pred_xgb_val))
r2_xgb = r2_score(y_val, y_pred_xgb_val)

print("\nResultados XGBoost (Validação):")
print(f"MAE: {mae_xgb:.4f} kW")
print(f"RMSE: {rmse_xgb:.4f} kW")
print(f"R²: {r2_xgb:.4f}")

In [ ]:
#═══════════════════════════════════════════════════════════════
# ETAPA 12: TREINAMENTO - MODELO 3: LIGHTGBM
#═══════════════════════════════════════════════════════════════

print("Treinando LightGBM Regressor...")

# Inicializar modelo
modelo_lgb = lgb.LGBMRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1,
    verbose=-1  # Suprimir mensagens de progresso
)

# Treinar
modelo_lgb.fit(X_treino_scaled, y_treino)

# Predições
y_pred_lgb_val = modelo_lgb.predict(X_val_scaled)

# Métricas no conjunto de validação
mae_lgb = mean_absolute_error(y_val, y_pred_lgb_val)
rmse_lgb = np.sqrt(mean_squared_error(y_val, y_pred_lgb_val))
r2_lgb = r2_score(y_val, y_pred_lgb_val)

print("\nResultados LightGBM (Validação):")
print(f"MAE: {mae_lgb:.4f} kW")
print(f"RMSE: {rmse_lgb:.4f} kW")
print(f"R²: {r2_lgb:.4f}")

In [ ]:
#═══════════════════════════════════════════════════════════════
# ETAPA 13: COMPARAÇÃO E SELEÇÃO DO MELHOR MODELO
#═══════════════════════════════════════════════════════════════

print("Comparando performance dos modelos...")

# Criar DataFrame com resultados
resultados = pd.DataFrame({
    'Modelo': ['Random Forest', 'XGBoost', 'LightGBM'],
    'MAE': [mae_rf, mae_xgb, mae_lgb],
    'RMSE': [rmse_rf, rmse_xgb, rmse_lgb],
    'R²': [r2_rf, r2_xgb, r2_lgb]
})

print("\nTabela Comparativa:")
print(resultados.to_string(index=False))

# Selecionar melhor modelo (menor MAE)
melhor_idx = resultados['MAE'].idxmin()
melhor_modelo_nome = resultados.loc[melhor_idx, 'Modelo']

print(f"\nMelhor modelo: {melhor_modelo_nome}")

# Atribuir o melhor modelo para uso posterior
if melhor_modelo_nome == 'Random Forest':
    modelo_final = modelo_rf
    y_pred_final = y_pred_rf_val
elif melhor_modelo_nome == 'XGBoost':
    modelo_final = modelo_xgb
    y_pred_final = y_pred_xgb_val
else:
    modelo_final = modelo_lgb
    y_pred_final = y_pred_lgb_val

# Visualização comparativa
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

modelos = ['Random Forest', 'XGBoost', 'LightGBM']
maes = [mae_rf, mae_xgb, mae_lgb]
r2s = [r2_rf, r2_xgb, r2_lgb]

axes[0].bar(modelos, maes, color=['#3498db', '#e74c3c', '#2ecc71'])
axes[0].set_title('MAE por Modelo')
axes[0].set_ylabel('MAE (kW)')
axes[0].set_ylim(0, max(maes) * 1.2)

axes[1].bar(modelos, r2s, color=['#3498db', '#e74c3c', '#2ecc71'])
axes[1].set_title('R² por Modelo')
axes[1].set_ylabel('R²')
axes[1].set_ylim(0, 1)

# Real vs Predito para o melhor modelo
axes[2].scatter(y_val, y_pred_final, alpha=0.5, s=10)
axes[2].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)
axes[2].set_xlabel('Geração Real (kW)')
axes[2].set_ylabel('Geração Predita (kW)')
axes[2].set_title(f'Real vs Predito - {melhor_modelo_nome}')

plt.tight_layout()
plt.show()

In [ ]:
#═══════════════════════════════════════════════════════════════
# ETAPA 14: SISTEMA DE LÓGICA FUZZY
#═══════════════════════════════════════════════════════════════

print("Criando sistema de Lógica Fuzzy...")

# Variável de entrada 1: Irradiação Solar (W/m²)
irradiacao_fuzzy = ctrl.Antecedent(np.arange(0, 1201, 1), 'irradiacao')
irradiacao_fuzzy['baixa'] = fuzz.trimf(irradiacao_fuzzy.universe, [0, 0, 400])
irradiacao_fuzzy['media'] = fuzz.trimf(irradiacao_fuzzy.universe, [300, 600, 900])
irradiacao_fuzzy['alta'] = fuzz.trimf(irradiacao_fuzzy.universe, [800, 1200, 1200])

# Variável de entrada 2: Temperatura Ambiente (°C)
temperatura_fuzzy = ctrl.Antecedent(np.arange(0, 51, 1), 'temperatura')
temperatura_fuzzy['normal'] = fuzz.trimf(temperatura_fuzzy.universe, [0, 0, 30])
temperatura_fuzzy['elevada'] = fuzz.trimf(temperatura_fuzzy.universe, [25, 35, 45])
temperatura_fuzzy['critica'] = fuzz.trimf(temperatura_fuzzy.universe, [40, 50, 50])

# Variável de entrada 3: kWh Previsto
kwh_fuzzy = ctrl.Antecedent(np.arange(0, 11, 0.1), 'kwh_previsto')
kwh_fuzzy['baixo'] = fuzz.trimf(kwh_fuzzy.universe, [0, 0, 3])
kwh_fuzzy['medio'] = fuzz.trimf(kwh_fuzzy.universe, [2, 5, 8])
kwh_fuzzy['alto'] = fuzz.trimf(kwh_fuzzy.universe, [7, 10, 10])

# Variável de saída: Eficiência (0-100)
eficiencia_fuzzy = ctrl.Consequent(np.arange(0, 101, 1), 'eficiencia')
eficiencia_fuzzy['critica'] = fuzz.trimf(eficiencia_fuzzy.universe, [0, 0, 40])
eficiencia_fuzzy['sub_otima'] = fuzz.trimf(eficiencia_fuzzy.universe, [30, 50, 70])
eficiencia_fuzzy['otima'] = fuzz.trimf(eficiencia_fuzzy.universe, [60, 100, 100])

# Regras Fuzzy
regra1 = ctrl.Rule(irradiacao_fuzzy['alta'] & temperatura_fuzzy['normal'], eficiencia_fuzzy['otima'])
regra2 = ctrl.Rule(irradiacao_fuzzy['alta'] & temperatura_fuzzy['elevada'], eficiencia_fuzzy['sub_otima'])
regra3 = ctrl.Rule(irradiacao_fuzzy['media'] & temperatura_fuzzy['normal'], eficiencia_fuzzy['sub_otima'])
regra4 = ctrl.Rule(irradiacao_fuzzy['media'] & temperatura_fuzzy['elevada'], eficiencia_fuzzy['sub_otima'])
regra5 = ctrl.Rule(irradiacao_fuzzy['baixa'] | temperatura_fuzzy['critica'], eficiencia_fuzzy['critica'])
regra6 = ctrl.Rule(kwh_fuzzy['alto'] & temperatura_fuzzy['normal'], eficiencia_fuzzy['otima'])
regra7 = ctrl.Rule(kwh_fuzzy['baixo'], eficiencia_fuzzy['critica'])
regra8 = ctrl.Rule(kwh_fuzzy['medio'] & temperatura_fuzzy['elevada'], eficiencia_fuzzy['sub_otima'])

# Sistema de controle fuzzy
sistema_eficiencia = ctrl.ControlSystem([regra1, regra2, regra3, regra4, regra5, regra6, regra7, regra8])
simulador_eficiencia = ctrl.ControlSystemSimulation(sistema_eficiencia)

print("Sistema Fuzzy criado com 8 regras!")
print("\nRegras implementadas:")
print("1. Irradiação ALTA + Temperatura NORMAL = Eficiência ÓTIMA")
print("2. Irradiação ALTA + Temperatura ELEVADA = Eficiência SUB-ÓTIMA")
print("3. Irradiação MÉDIA + Temperatura NORMAL = Eficiência SUB-ÓTIMA")
print("4. Irradiação MÉDIA + Temperatura ELEVADA = Eficiência SUB-ÓTIMA")
print("5. Irradiação BAIXA OU Temperatura CRÍTICA = Eficiência CRÍTICA")
print("6. kWh ALTO + Temperatura NORMAL = Eficiência ÓTIMA")
print("7. kWh BAIXO = Eficiência CRÍTICA")
print("8. kWh MÉDIO + Temperatura ELEVADA = Eficiência SUB-ÓTIMA")

In [ ]:
#═══════════════════════════════════════════════════════════════
# ETAPA 15: FUNÇÃO DE CONSULTA E SISTEMA DE RECOMENDAÇÕES
#═══════════════════════════════════════════════════════════════

def consultar_previsao(data_hora_str, mostrar_detalhes=True):
    """
    Consulta previsão de geração, classificação fuzzy e recomendações

    Parâmetros:
    - data_hora_str: string no formato 'YYYY-MM-DD HH:MM:SS'
    - mostrar_detalhes: bool, se True mostra informações completas

    Retorna: dict com todas as informações
    """

    # Converter string para datetime
    data_hora_consulta = pd.to_datetime(data_hora_str)

    # Buscar registro mais próximo no dataset
    df_limpo['diff_tempo'] = abs(df_limpo['data_hora'] - data_hora_consulta)
    idx_mais_proximo = df_limpo['diff_tempo'].idxmin()
    registro = df_limpo.loc[idx_mais_proximo]

    # Preparar dados para previsão
    features_registro = registro[X.columns]
    features_scaled = scaler.transform([features_registro])

    # PREVISÃO DE GERAÇÃO
    geracao_prevista = modelo_final.predict(features_scaled)[0]

    # DADOS CLIMÁTICOS
    irradiacao_real = registro['irradiacao']
    temperatura_real = registro['temperatura_ambiente']

    # CLASSIFICAÇÃO FUZZY
    try:
        simulador_eficiencia.input['irradiacao'] = min(1200, max(0, irradiacao_real))
        simulador_eficiencia.input['temperatura'] = min(50, max(0, temperatura_real))
        simulador_eficiencia.input['kwh_previsto'] = min(10, max(0, geracao_prevista))
        simulador_eficiencia.compute()

        eficiencia_score = simulador_eficiencia.output['eficiencia']

        # Classificar categoria
        if eficiencia_score >= 70:
            categoria_eficiencia = "OTIMA"
        elif eficiencia_score >= 40:
            categoria_eficiencia = "SUB-OTIMA"
        else:
        categoria_eficiencia = "CRITICA"
except:
    eficiencia_score = 50
    categoria_eficiencia = "SUB-OTIMA"

# GERAR RECOMENDAÇÕES
consumo_estimado = 5.0  # kW (valor médio, pode ser ajustado)
excedente = geracao_prevista - consumo_estimado
tarifa_kwh = 0.75  # R$ por kWh

# Recomendações por categoria
if categoria_eficiencia == "OTIMA" and excedente > 2:
    status = "EXCEDENTE ALTO"
    recomendacoes = [
        f"Voce vai gerar {geracao_prevista:.2f} kW",
        f"Consumo estimado: {consumo_estimado:.2f} kW",
        f"EXCEDENTE: {excedente:.2f} kW",
        "ACOES RECOMENDADAS:",
        "- Armazene em baterias para usar a noite",
        "- Venda o excedente para a rede eletrica",
        "- Aproveite para usar aparelhos pesados",
        f"Economia estimada: R$ {excedente * tarifa_kwh:.2f}/hora"
    ]
elif categoria_eficiencia == "SUB-OTIMA":
    status = "GERACAO MODERADA"
    if excedente > 0:
        recomendacoes = [
            f"Geracao prevista: {geracao_prevista:.2f} kW",
            f"Excedente pequeno: {excedente:.2f} kW",
            "ACOES RECOMENDADAS:",
            "- Use energia conscientemente",
            "- Evite muitos aparelhos simultaneos",
            "- Priorize aparelhos essenciais"
        ]
    else:
        recomendacoes = [
            f"Geracao prevista: {geracao_prevista:.2f} kW",
            f"Deficit: {abs(excedente):.2f} kW",
            "ACOES RECOMENDADAS:",
            "- Use bateria armazenada se disponivel",
            "- Reduza consumo de aparelhos pesados",
            f"Custo extra da rede: R$ {abs(excedente) * tarifa_kwh:.2f}/hora"
        ]
else:  # CRITICA
    status = "GERACAO BAIXA - ALERTA"
    recomendacoes = [
        f"Geracao prevista: apenas {geracao_prevista:.2f} kW",
        f"DEFICIT: {abs(excedente):.2f} kW",
        "ACAO NECESSARIA:",
        "- Use energia da bateria",
        "- Adie tarefas pesadas",
        "- DESLIGUE: ar-condicionado, aquecedores",
        f"Custo da rede: R$ {abs(excedente) * tarifa_kwh:.2f}/hora"
    ]

    # Detectar possível problema técnico
    if irradiacao_real > 600 and geracao_prevista < 3:
        recomendacoes.append("ATENCAO: Sol forte mas geracao baixa!")
        recomendacoes.append("- Verificar paineis sujos ou sombreamento")

# Criar dicionário de resultados
resultado = {
    'data_hora': registro['data_hora'],
    'irradiacao': irradiacao_real,
    'temperatura': temperatura_real,
    'geracao_prevista': geracao_prevista,
    'eficiencia_score': eficiencia_score,
    'categoria_eficiencia': categoria_eficiencia,
    'status': status,
    'recomendacoes': recomendacoes
}

# Mostrar detalhes se solicitado
if mostrar_detalhes:
    print("="*70)
    print(f"CONSULTA: {data_hora_str}")
    print("="*70)
    print(f"\nDADOS CLIMATICOS:")
    print(f"  Irradiacao Solar: {irradiacao_real:.2f} W/m2")
    print(f"  Temperatura Ambiente: {temperatura_real:.2f} C")
    print(f"\nPREVISAO DE GERACAO:")
    print(f"  Geracao Prevista: {geracao_prevista:.2f} kW")
    print(f"  Equivalente a: {geracao_prevista * 1:.2f} kWh em 1 hora")
    print(f"\nCLASSIFICACAO FUZZY:")
    print(f"  Eficiencia: {categoria_eficiencia} ({eficiencia_score:.1f}/100)")
    print(f"\nRECOMENDACOES:")
    print(f"  STATUS: {status}")
    for rec in recomendacoes:
        print(f"  {rec}")
    print("="*70)

return resultado

In [ ]:
#═══════════════════════════════════════════════════════════════
# ETAPA 16: TESTE DA FUNÇÃO DE CONSULTA
#═══════════════════════════════════════════════════════════════

# Testar com uma data do dataset
data_teste = df_limpo['data_hora'].iloc[len(df_limpo)//2].strftime('%Y-%m-%d %H:%M:%S')

print(f"Testando consulta para: {data_teste}\n")

resultado_teste = consultar_previsao(data_teste)


In [ ]:
#═══════════════════════════════════════════════════════════════
# ETAPA 17: PAINEL DE CONTROLE - PREVISAO DIA COMPLETO
#═══════════════════════════════════════════════════════════════

def painel_dia_completo(data_str):
    """
    Mostra previsões para as 24 horas de um dia específico

    Parâmetros:
    - data_str: string no formato 'YYYY-MM-DD'
    """

    print("="*80)
    print(f"PAINEL DE CONTROLE - DIA COMPLETO: {data_str}")
    print("="*80)

    resultados_dia = []

    for hora in range(24):
        data_hora_str = f"{data_str} {hora:02d}:00:00"

        try:
            resultado = consultar_previsao(data_hora_str, mostrar_detalhes=False)

            # Simplificar categoria
            if resultado['categoria_eficiencia'] == 'OTIMA':
                emoji_status = 'OTIMA'
            elif resultado['categoria_eficiencia'] == 'SUB-OTIMA':
                emoji_status = 'MEDIA'
            else:
                emoji_status = 'BAIXA'

            resultados_dia.append({
                'Hora': f"{hora:02d}:00",
                'Geracao_kW': f"{resultado['geracao_prevista']:.2f}",
                'Eficiencia': emoji_status,
                'Score': f"{resultado['eficiencia_score']:.0f}",
                'Irradiacao': f"{resultado['irradiacao']:.0f}",
                'Temp_C': f"{resultado['temperatura']:.1f}"
            })
        except:
            continue

    # Criar DataFrame
    df_painel = pd.DataFrame(resultados_dia)

    # Mostrar tabela
    print("\n")
    print(df_painel.to_string(index=False))

    # Calcular resumo do dia
    if len(resultados_dia) > 0:
        geracoes = [float(r['Geracao_kW']) for r in resultados_dia]
        total_dia = sum(geracoes)
        media_hora = np.mean(geracoes)
        pico = max(geracoes)
        hora_pico = geracoes.index(pico)

        print("\n" + "="*80)
        print("RESUMO DO DIA:")
        print(f"  Total gerado: {total_dia:.2f} kWh")
        print(f"  Media por hora: {media_hora:.2f} kW")
        print(f"  Pico de geracao: {pico:.2f} kW as {hora_pico:02d}:00")
        print(f"  Valor economizado: R$ {total_dia * 0.75:.2f}")
        print("="*80)

        # Gráfico do dia
        plt.figure(figsize=(12, 5))
        plt.plot(range(len(geracoes)), geracoes, marker='o', linewidth=2, markersize=6)
        plt.fill_between(range(len(geracoes)), geracoes, alpha=0.3)
        plt.xlabel('Hora do Dia')
        plt.ylabel('Geracao Prevista (kW)')
        plt.title(f'Previsao de Geracao - {data_str}')
        plt.xticks(range(0, 24, 2), [f'{h:02d}:00' for h in range(0, 24, 2)])
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

print("Funcao de painel criada com sucesso!")
print("\nExemplo de uso:")
print("painel_dia_completo('2023-06-15')")

In [ ]:
#═══════════════════════════════════════════════════════════════
# ETAPA 18: TESTE DO PAINEL DE CONTROLE
#═══════════════════════════════════════════════════════════════

# Selecionar uma data do dataset para testar
data_exemplo = df_limpo['data_hora'].iloc[1000].strftime('%Y-%m-%d')

print(f"Gerando painel para o dia: {data_exemplo}\n")

painel_dia_completo(data_exemplo)

In [ ]:
#═══════════════════════════════════════════════════════════════
# ETAPA 19: VISUALIZAÇÕES FINAIS E INSIGHTS
#═══════════════════════════════════════════════════════════════

print("Gerando visualizacoes finais...")

# 1. Feature Importance do melhor modelo
if hasattr(modelo_final, 'feature_importances_'):
    importancias = modelo_final.feature_importances_
    indices = np.argsort(importancias)[::-1][:10]  # Top 10

    plt.figure(figsize=(10, 6))
    plt.barh(range(10), importancias[indices])
    plt.yticks(range(10), [X.columns[i] for i in indices])
    plt.xlabel('Importancia')
    plt.title(f'Top 10 Features Mais Importantes - {melhor_modelo_nome}')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

# 2. Análise de Resíduos
residuos = y_val - y_pred_final

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma dos resíduos
axes[0].hist(residuos, bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(0, color='red', linestyle='--', linewidth=2)
axes[0].set_xlabel('Residuos (kW)')
axes[0].set_ylabel('Frequencia')
axes[0].set_title('Distribuicao dos Residuos')

# Resíduos vs Valores Preditos
axes[1].scatter(y_pred_final, residuos, alpha=0.5, s=10)
axes[1].axhline(0, color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('Valores Preditos (kW)')
axes[1].set_ylabel('Residuos (kW)')
axes[1].set_title('Residuos vs Valores Preditos')

plt.tight_layout()
plt.show()

print("Visualizacoes concluidas!")

In [ ]:
#═══════════════════════════════════════════════════════════════
# ETAPA 20: AVALIAÇÃO FINAL NO CONJUNTO DE TESTE
#═══════════════════════════════════════════════════════════════

print("Avaliando modelo final no conjunto de TESTE...")

# Predições no conjunto de teste
y_pred_teste = modelo_final.predict(X_teste_scaled)

# Métricas finais
mae_teste = mean_absolute_error(y_teste, y_pred_teste)
rmse_teste = np.sqrt(mean_squared_error(y_teste, y_pred_teste))
r2_teste = r2_score(y_teste, y_pred_teste)
mape_teste = np.mean(np.abs((y_teste - y_pred_teste) / y_teste)) * 100

print("\nRESULTADOS FINAIS NO CONJUNTO DE TESTE:")
print("="*70)
print(f"Modelo: {melhor_modelo_nome}")
print(f"MAE: {mae_teste:.4f} kW")
print(f"RMSE: {rmse_teste:.4f} kW")
print(f"R²: {r2_teste:.4f}")
print(f"MAPE: {mape_teste:.2f}%")
print("="*70)

# Gráfico Real vs Predito no teste
plt.figure(figsize=(10, 6))
plt.scatter(y_teste, y_pred_teste, alpha=0.5, s=20)
plt.plot([y_teste.min(), y_teste.max()], [y_teste.min(), y_teste.max()], 'r--', lw=3, label='Linha Ideal')
plt.xlabel('Geracao Real (kW)')
plt.ylabel('Geracao Predita (kW)')
plt.title(f'Real vs Predito - Conjunto de Teste - {melhor_modelo_nome}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
#═══════════════════════════════════════════════════════════════
# INFORMAÇÕES DO PROJETO
#═══════════════════════════════════════════════════════════════

print("="*80)
print("PROJETO: PREVISAO DE PRODUTIVIDADE DE PAINEIS SOLARES")
print("="*80)
print("\nINTEGRANTES:")
print("  - Demerson Sampaio (12725235937)")
print("  - Josue Serra (12724147248)")
print("  - Rian Assis (1272418628)")
print("  - Sabrina Oliveira (12725234634)")
print("\nOBJETIVOS:")
print("  1. Prever geracao de energia nas proximas 24 horas")
print("  2. Classificar eficiencia dos paineis (Otima/Sub-otima/Critica)")
print("  3. Gerar recomendacoes inteligentes ao usuario")
print("\nTECNOLOGIAS UTILIZADAS:")
print("  - Python 3.x")
print("  - scikit-learn (Machine Learning)")
print("  - XGBoost e LightGBM (Modelos avancados)")
print("  - scikit-fuzzy (Logica Fuzzy)")
print("  - pandas e numpy (Manipulacao de dados)")
print("  - matplotlib e seaborn (Visualizacoes)")
print("\nDATASETS:")
print(f"  - Total de registros processados: {len(df_limpo):,}")
print(f"  - Periodo: {df_limpo['data_hora'].min()} ate {df_limpo['data_hora'].max()}")
print(f"  - Features criadas: {len(X.columns)}")
print("\nRESULTADOS:")
print(f"  - Melhor modelo: {melhor_modelo_nome}")
print(f"  - R² no teste: {r2_teste:.4f} ({r2_teste*100:.2f}% da variancia explicada)")
print(f"  - Erro medio: {mae_teste:.4f} kW")
print(f"  - Sistema Fuzzy: 8 regras implementadas")
print("="*80)
